In [ ]:
# review -> 이모티콘/자음 삭제 -> (띄워쓰기 + 맞춤법) -> 문장 분리
# - 문장 단위: 문장 부호, 종결어미, 연결조사로 사전 만들기

In [21]:
import pandas as pd
from kiwipiepy import Kiwi
from emoji import core
import re

In [5]:
data = pd.read_csv('./data/train_data_1.csv', encoding='utf-8-sig')
data.head()

,review
0,맛있어요~
1,만족도 최상의 커피맛집
2,찐 커피맛집ㅎㅎ
3,로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마...
4,커조남 커피는 힐링입니다 소금빵도 맛있어요


In [2]:
kiwi = Kiwi()

In [22]:
# 사용자 사전 추가
kiwi.add_user_word('맛집', 'NNG', 0)
kiwi.add_user_word('카페명', 'NNP', 0) # 카페 이름 리스트가 있으면 추가

True

In [23]:
# 정규표현식 컴파일 (한글 자음 미포함)
ascii_patterns = re.compile(r'[^\w\s]')

In [40]:
def cleanReview(review):
    sent = core.replace_emoji(review, replace="")
    out = sent.replace("^^", "")
    return out

In [38]:
def revToSent(review):
    sentences = []
    sents = kiwi.split_into_sents(review, return_tokens=True)
    for sent in sents:
        toks = sent.tokens
        tmp = []
        for tok in toks:
            tmp.append(tok)
            
            if tok.tag == 'EC' and tok.form == '지만':
                sentences.append(kiwi.join(tmp))
                tmp.clear()
            
            if tok.tag == 'SO' and tok.form == '~':
                sentences.append(kiwi.join(tmp))
                tmp.clear()

        if len(tmp) > 0:
            sentences.append(kiwi.join(tmp))
    
    return sentences

In [26]:
s = '모르시는 대구분들 계신가용~?'
cleanReview(s)

'모르시는 대구분들 계신가용'

In [43]:
for i in range(3, 5):
    review = data['review'][i]
    print('원본 문장: ', review)
    re_wo_emoji = core.replace_emoji(review, replace="")
    sents = kiwi.split_into_sents(re_wo_emoji, return_tokens=True)
    print('문장 분리: ', sents)
    print()    

원본 문장:  로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마리토쪼도 크림 부드럽고 빵도 포실포실🥰 재방문 의사 있어요
문장 분리:  [Sentence(text='로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!!', start=0, end=44, tokens=[Token(form='로마노', tag='NNP', start=0, len=3), Token(form='진짜', tag='MAG', start=4, len=2), Token(form='달달하', tag='VA', start=7, len=3), Token(form='고', tag='EC', start=10, len=1), Token(form='맛있', tag='VA', start=12, len=2), Token(form='고', tag='EC', start=14, len=1), Token(form='아이스크림', tag='NNG', start=16, len=5), Token(form='라', tag='NNG', start=21, len=1), Token(form='떼', tag='NNG', start=22, len=1), Token(form='도', tag='JX', start=23, len=1), Token(form='꼬소', tag='NNG', start=25, len=2), Token(form='하', tag='XSA', start=27, len=1), Token(form='고', tag='EC', start=28, len=1), Token(form='달달하', tag='VA', start=30, len=3), Token(form='고', tag='EC', start=33, len=1), Token(form='진짜', tag='MAG', start=35, len=2), Token(form='맛있', tag='VA', start=38, len=2), Token(form='어요', tag='EF', start=40, len=2), Toke

In [42]:
for i in range(5):
    review = data['review'][i]
    print('원본 문장: ', review)
    re_wo_emoji = cleanReview(review)
    # re_wo_emoji = core.replace_emoji(review, replace="")
    print('특수 문자 제거: ', re_wo_emoji)
    sents = revToSent(re_wo_emoji)
    print('문장 분리: ', sents)
    print()    

원본 문장:  맛있어요~
특수 문자 제거:  맛있어요~
문장 분리:  ['맛있어요~']

원본 문장:  만족도 최상의 커피맛집
특수 문자 제거:  만족도 최상의 커피맛집
문장 분리:  ['만족도 최상의 커피 맛집']

원본 문장:  찐 커피맛집ㅎㅎ
특수 문자 제거:  찐 커피맛집ㅎㅎ
문장 분리:  ['찐 커피 맛집 ㅎㅎ']

원본 문장:  로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마리토쪼도 크림 부드럽고 빵도 포실포실🥰 재방문 의사 있어요
특수 문자 제거:  로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마리토쪼도 크림 부드럽고 빵도 포실포실 재방문 의사 있어요
문장 분리:  ['로마노 진짜 달달하고 맛있고 아이스크림 라 떼도 꼬소하고 달달하고 진짜 맛있어요!!', '마리토쪼도 크림 부드럽고 빵도 포실포실 재방문 의사 있어요']

원본 문장:  커조남 커피는 힐링입니다 소금빵도 맛있어요
특수 문자 제거:  커조남 커피는 힐링입니다 소금빵도 맛있어요
문장 분리:  ['커조남 커피는 힐링입니다', '소금 빵도 맛있어요']



In [8]:


for review in data['review']:
    sentence = kiwi.split_into_sents(data['review'][i])
    print(sentence)

[Sentence(text='시나몬롤 맛있어용 ✨', start=0, end=11, tokens=None, subs=[]), Sentence(text='하지만 소금빵은 동굴이 없고 좀 말라있어서 아쉬웠어요', start=12, end=41, tokens=None, subs=[])]
[Sentence(text='드립커피하면 커조남!', start=0, end=11, tokens=None, subs=[]), Sentence(text='오늘 드립커피는 좀 애매한 맛이긴 했지만 리필커피가 맛있어서 괜춘~바로 앞 공영주차장에 주차가능', start=12, end=65, tokens=None, subs=[])]
[Sentence(text='최애 커피집입니다 ㅎㅎ', start=0, end=12, tokens=None, subs=[]), Sentence(text='앞산에 있을 때 부터 좋아하던 곳이라 계속가게되네요', start=13, end=41, tokens=None, subs=[]), Sentence(text='크리스마스페어에서도 로마노 한잔~^^ 더웠는데 시원하게 상큼하게 잘마셨습니다', start=43, end=85, tokens=None, subs=[])]
[Sentence(text='커피맛집👍', start=0, end=5, tokens=None, subs=[])]
[Sentence(text='커피맛을 조금 아는 남자가 반갑게 맞아주는\n찐 커피 맛집!!', start=0, end=33, tokens=None, subs=[])]
